In [32]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, Button, HoverTool, Select
from bokeh.palettes import brewer
from bokeh.layouts import layout, widgetbox, row, column
from bokeh.models.glyphs import Patches

import holoviews as hv
import os

In [33]:
import sys,os

### Wczytanie danych

* Dane o gestości zaludnienia na 1km^2 w woj. mazowieckim w latach 2008-2018, GUS: 
https://bdl.stat.gov.pl/BDL/dane/podgrup/wymiary
* Dane o położeniu gmin w Polsce: http://www.gugik.gov.pl/pzgik/dane-bez-oplat/dane-z-panstwowego-rejestru-granic-i-powierzchni-jednostek-podzialow-terytorialnych-kraju-prg (jednostki_administracyjne.zip --> pliki .shp)

Pliki znajdują się w folderze ../data/jednostki_administracyjne/ i ../data/dane_gus/

bokeh na podstawie: https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
opcja play (HoloViews) na podstawie: http://holoviews.org/reference/apps/bokeh/player.html

In [34]:
def TERYT_powiat(x):
    if len(str(x))>=4:     
        if str(x)[3]=='0': 
            return '0' + str(x)[0:3] 
        else: 
            return str(x)[0:4]
    else: return str(x)[0:4]

# Powiaty - geolokalizacja

In [35]:
shapefile = os.path.realpath('jednostki_administracyjne/Powiaty.shp')
#Read shapefile using Geopandas
pdf = gpd.read_file(shapefile,  encoding='utf-8')[['JPT_KOD_JE', 'JPT_NAZWA_', 'geometry']]
pdf.columns = ['TERYT', 'powiat', 'geometry']
pdf.TERYT = pdf.TERYT.apply(lambda x: TERYT_powiat(x) ) ### !!!!! pierwsze 4 (powiat)

In [60]:
pdf[pdf.powiat.apply(lambda x: 'żag' in x)]

,TERYT,powiat,geometry
106,0081,powiat żagański,"POLYGON ((15.10533765400004 51.42730160700006,..."


## Połączenie zbiorów

### Powiaty - dane z GUS

In [109]:
dane_p = pd.DataFrame()
zb = 'wynagrodzenie_srednie_brutto'
path = 'dane_gus/powiaty/' + zb + '.csv'
datafile = os.path.realpath(path) 
#Read csv file using pandas
df = pd.read_csv(datafile, sep=';', decimal=',')
df.columns=['TERYT', 'Nazwa', zb +'_' + '2008',  zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', zb +'_' + '2018', '']
df = df.drop(columns='')
df.TERYT =df.TERYT.apply(lambda x: TERYT_powiat(x) ) ### !!!!! pierwsze 4 (powiat)

dane_p = df   

In [110]:
zbiory = [ 'nowo_zarejestr_podm_sekt_pryw', 
          'nowe_sekt_pryw_sp_handlowe', 
          'nowe_sekt_publ_sp_handlowe', 
          'podm_w_rej_reg_na10tys_ludn', 
          'udzial_wyrejestr_do_nowych', 
          'podm_w_rej_reg_sekt_pryw_HANDL', 
          'bud_niemieszk_ogolem', 
          'magazyny_silosy_zb',
         'drogi_utwardzone_km', 
         'drogi_utwardzone_km_na100km2', 
         'gestosc_zaludnienia_na1km2']
   

In [111]:
for zb in zbiory:
    path = 'dane_gus/powiaty/' + zb + '.csv'
    datafile = os.path.realpath(path) 
    #Read csv file using pandas
    df = pd.read_csv(datafile, sep=';', decimal=',')
    if len(df.columns) == 13: #drogi_utwardzone_km
        df.columns=['TERYT', 'Nazwa', zb +'_' + '2008',  zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', '']
    else:
        df.columns=['TERYT', 'Nazwa', zb +'_' + '2008',  zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', zb +'_' + '2018', '']
    df = df.drop(columns='')
    df.TERYT =df.TERYT.apply(lambda x: TERYT_powiat(x))   
   
    dane_p = pd.merge(dane_p, df, on=['TERYT', 'Nazwa'], how='outer')

In [112]:
dane_p.head(10)

,TERYT,Nazwa,wynagrodzenie_srednie_brutto_2008,wynagrodzenie_srednie_brutto_2009,wynagrodzenie_srednie_brutto_2010,wynagrodzenie_srednie_brutto_2011,wynagrodzenie_srednie_brutto_2012,wynagrodzenie_srednie_brutto_2013,wynagrodzenie_srednie_brutto_2014,wynagrodzenie_srednie_brutto_2015,...,drogi_utwardzone_km_2008,drogi_utwardzone_km_2009,drogi_utwardzone_km_2010,drogi_utwardzone_km_2011,drogi_utwardzone_km_2012,drogi_utwardzone_km_2013,drogi_utwardzone_km_2014,drogi_utwardzone_km_2015,drogi_utwardzone_km_2016,drogi_utwardzone_km_2017
0,0,POLSKA,3158.48,3315.38,3435.00,3625.21,3744.38,3877.43,4003.99,4150.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0200,DOLNOŚLĄSKIE,3135.83,3295.44,3412.37,3587.25,3709.32,3868.86,4042.86,4204.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0201,Powiat bolesławiecki,2476.41,2629.01,2789.80,2936.92,3057.28,3231.49,3422.90,3548.69,...,297.7,297.7,298.5,298.5,298.5,298.5,298.5,298.5,298.5,314.1
3,0202,Powiat dzierżoniowski,2489.40,2645.13,2832.05,3024.11,3113.13,3156.11,3299.96,3406.18,...,213.0,200.0,200.0,200.2,200.2,200.2,200.2,200.2,200.5,200.2
4,0203,Powiat głogowski,2647.38,2859.95,2914.21,3028.65,3122.99,3320.48,3460.12,3615.99,...,124.3,124.3,124.3,124.3,124.3,102.0,121.6,119.6,119.6,119.6
5,0204,Powiat górowski,2376.54,2464.05,2598.06,2755.49,2908.49,3066.83,3142.52,3242.71,...,286.0,282.2,282.2,282.2,282.2,267.1,282.2,282.2,282.2,266.6
6,0205,Powiat jaworski,2368.60,2506.86,2654.67,2837.23,2983.89,3040.71,3142.45,3323.37,...,306.0,305.0,305.0,305.0,304.0,304.0,304.0,304.0,300.4,300.4
7,0206,Powiat jeleniogórski,2369.63,2562.15,2699.92,2867.09,3031.43,3128.87,3232.50,3413.66,...,264.5,264.5,264.5,264.5,257.3,257.3,257.3,240.9,240.9,240.9
8,0207,Powiat kamiennogórski,2374.82,2546.98,2701.25,2864.01,2983.93,3045.35,3161.57,3269.22,...,198.1,198.1,198.1,198.1,198.1,198.1,198.1,198.1,202.6,202.6
9,0208,Powiat kłodzki,2511.35,2663.41,2803.23,2962.33,3063.43,3200.48,3333.36,3410.23,...,696.9,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3,696.3


In [64]:
dane_p[dane_p.Nazwa.apply(lambda x: 'żeg' in x)]

,TERYT,Nazwa,wynagrodzenie_srednie_brutto_2008,wynagrodzenie_srednie_brutto_2009,wynagrodzenie_srednie_brutto_2010,wynagrodzenie_srednie_brutto_2011,wynagrodzenie_srednie_brutto_2012,wynagrodzenie_srednie_brutto_2013,wynagrodzenie_srednie_brutto_2014,wynagrodzenie_srednie_brutto_2015,...,nowo_zarejestr_podm_sekt_pryw_2009,nowo_zarejestr_podm_sekt_pryw_2010,nowo_zarejestr_podm_sekt_pryw_2011,nowo_zarejestr_podm_sekt_pryw_2012,nowo_zarejestr_podm_sekt_pryw_2013,nowo_zarejestr_podm_sekt_pryw_2014,nowo_zarejestr_podm_sekt_pryw_2015,nowo_zarejestr_podm_sekt_pryw_2016,nowo_zarejestr_podm_sekt_pryw_2017,nowo_zarejestr_podm_sekt_pryw_2018


### obsluga wyjatkow

In [43]:
dane_p.loc[dane_p.TERYT == '0620', 'TERYT'] = '0062'
dane_p.loc[dane_p.TERYT == '0610', 'TERYT'] = '0061'
dane_p.loc[dane_p.TERYT == '0410', 'TERYT'] = '0041'
dane_p.loc[dane_p.TERYT == '0210', 'TERYT'] = '0021'
dane_p.loc[dane_p.TERYT == '0220', 'TERYT'] = '0022'
dane_p.loc[dane_p.TERYT == '0810', 'TERYT'] = '0081'

## Wizualizacja - bokeh

In [41]:
renderer = hv.renderer('bokeh')

In [42]:
# Declare the HoloViews object
start = 2008
end = 2018

In [ ]:
def animate_update():
    year = slider.value
    if year < end:
        year = slider.value + 1
        slider.value = year
            

def animate():
    if button.label == '► Play' and slider.value<end:
        button.label = '❚❚ Pause'
        curdoc().add_periodic_callback(animate_update, 10000) ## co 1000 mili sec
    else:
        print('$$$$$$$$$$$ zakonczone $$$$$$$$$$')
        button.label = '► Play'
        curdoc().remove_periodic_callback(animate_update)
        slider.value==2008

In [ ]:
#Define function that returns json_data for year selected by user.
    
def json_data():
    
    #yr = selectedYear
    #zb = selectedData
    #df = dane[dane.rodzaj_danych == zb]
    #df_yr = df[['TERYT', 'Nazwa', str(yr)]]
    #df_yr.columns = ['TERYT', 'Nazwa', 'wartosc']
    #print(df_yr.head(10))
    
    df = dane_p
    #Merge gus dataframes pdf 
    merged = pdf.set_index('TERYT').merge(df.set_index('TERYT'), on='TERYT', how='left') 
    
    print(merged.head(10))
    
    #merged = gdf.merge(df_yr, left_on = 'country_code', right_on =     'code', how = 'left')
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data())
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 6000, nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}


#Add hover tool
col='wynagrodzenie_srednie_brutto_2008'
hover = HoverTool(tooltips = [ ('powiat',' @powiat'),('wartosc', '@' + col)])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal') #, major_label_overrides = tick_labels)

#Create figure object.
p = figure(title ='Wynagrodzenie_srednie_brutto w 2008r.', plot_height = 1000 , plot_width = 1000, 
           toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :col, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify layout
p.add_layout(color_bar, 'below')
# Define the callback function: update_plot
def update_plot(attr, old, new):
    print('wybrany rok: ', slider.value)
    print('wybrany zb: ', select.value)
    yr = slider.value
    zb = select.value
    
    col = select.value+'_' + str(slider.value)
    hover = HoverTool(tooltips = [ ('powiat',' @powiat'),('wartosc', '@' + col)])
    p.add_tools(hover)

    p.patches('xs','ys', source = geosource,fill_color = {'field' :col, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
    
    if select.value == 'udzial_wyrejestr_do_nowych':
        p.title.text = 'Udział wyrejestrowanych podmiotów do zarejestrowanych [%] w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=20
        
    elif select.value == 'podm_w_rej_reg_na10tys_ludn':
        p.title.text = 'Liczba podmiotów wpisanych do rej. REGON na 10tys. ludności - stan na %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=3000
        
    elif select.value == 'nowe_sekt_publ_sp_handlowe':
        p.title.text =  'Liczba nowo zarejestr. podmiotów HANDLOWYCH w sekt. publicznym w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=10
    
    elif select.value == 'nowe_sekt_pryw_sp_handlowe':
        p.title.text = 'Liczba nowo zarejestr. podmiotów HANDLOWYCH w sekt. prywatnym w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=500
    
    elif select.value == 'nowo_zarejestr_podm_sekt_pryw':
        p.title.text = 'Liczba nowo zarejestrowanych podmiotow w sektorze prywatnym w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=3000
    
    
    elif select.value == 'wynagrodzenie_srednie_brutto':
        p.title.text = 'Wynagrodzenie_srednie_brutto w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=6000
        
    elif select.value == 'podm_w_rej_reg_sekt_pryw_HANDL':
        p.title.text = 'Podmioty zarejstrowane wg REGON, sekt. pryw. HANDLOWE, %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=15000   
        
        
    elif select.value ==  'bud_niemieszk_ogolem':
        p.title.text = 'Powierzchnia użytkowa budynków niemieszkalnych ogółem [m2], %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=300000   
        
    elif select.value ==  'magazyny_silosy_zb':
        p.title.text = 'Powierzchnia użytkowa magazynów/silosów/zbiorników ogółem [m2], oddane do użytkowania w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=200000
        
    elif select.value ==  'drogi_utwardzone_km':
        p.title.text = 'Łączna długość drug utwardzonych [km] , %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=1000   
   
    elif select.value ==  'drogi_utwardzone_km_na100km2':
            p.title.text = 'Łączna długość drug utwardzonych na 100km2,  [km] , %d r.' %yr
            # update the existing transform
            color_mapper.low=0
            color_mapper.high=400   
        
        
    elif select.value ==  'gestosc_zaludnienia_na1km2':
            p.title.text = 'Gęstość zaludnienia na 1km2, %d r.' %yr
            # update the existing transform
            color_mapper.low=0
            color_mapper.high=1000    
      
       

        
        
    #p.update(slider.value)
    
    
    color_bar.trigger('change')

    
# Make a slider object: slider 
slider = Slider(title = 'Rok',start = 2008, end = 2018, step = 1, value = 2008)
slider.on_change('value', update_plot)

# Make select box
select = Select(title="Prezentowana wartość:", value="wynagrodzenie_srednie_brutto", options=zbiory)
select.on_change('value', update_plot)

# Button for animation
button = Button(label='► Play', width=60)
button.on_click(animate)


# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider), widgetbox(select), widgetbox(button))
curdoc().add_root(layout)
#Display plot inline in Jupyter notebook
#output_notebook()
#Display plot
#show(layout)
